In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt

Get the links for today's markets

In [10]:
def get_links_SB(date=dt.date.today().strftime("%d/%m/%y"),
                chrome_path=r"C:\Users\johnn\chromedriver\chromedriver.exe",
                url="https://www.sportsbet.com.au/betting/basketball-us/nba-matches"):
    ''' 
    date in dd/mm/yy format
    Goes through sportsbet's NBA games page and returns a list of links to all open markets.
    Open meaning of the day specified and not in play
    '''

    driver = webdriver.Chrome(chrome_path)
    driver.get(url)

    games = driver.find_elements_by_class_name("link_ft4u1lp")
    
    # Filter for date's games that haven't started already
    today_games = [game for game in games if date in game.find_element_by_class_name('time_fbgyqei').text]
    
    links = [game.get_attribute('href') for game in today_games]
    driver.close()
    return links

Go through link and get player markets

In [18]:
def get_player_markets_SB(link,
                         chrome_path=r"C:\Users\johnn\chromedriver\chromedriver.exe"):
    '''
    Returns a list of strings parsed from sportsbets player markets.
    '''
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")

    link = links[0]

    driver = webdriver.Chrome(chrome_path, options=options)
    driver.get(link)

    markets = driver.find_elements_by_class_name("touchArea_fgteabt")

    markets_of_interest = ['Top Markets', # To minimise
                           'Player Points Markets', 
                           'Player Rebounds Markets', 
                           'Player Assists Markets']
    main_markets = [market for market in markets if market.text in markets_of_interest]

    for market in main_markets:
        market.click()
        time.sleep(1)

    # Find player markets e.g. Donovan Mitchell - Points
    # Must be maximised windows for class name accordionItemDesktop_f1pa6f05
    player_markets = driver.find_elements_by_class_name("accordionItemDesktop_f1pa6f05")

    player_markets_of_interest = ['- Points', '- Rebounds', '- Assists']
    main_player_markets = [market for market in player_markets if any(
        stat in market.text for stat in player_markets_of_interest)]

    # Scroll to top so that the first box isn't hidden by Sportsbet's ribbon
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
    time.sleep(1) # Wait for page to finish scrolling

    # Open all the markets to see odds and record them
    market_list = []
    for player_market in main_player_markets:
        player_market.click()
        time.sleep(0.5)

        market_list.append(player_market.text)
    
    driver.close()
    
    return market_list

In [13]:
links = get_links_SB('06/01/19')

found 8 games


In [19]:
player_markets = get_player_markets_SB(links[3])

In [23]:
player_markets

['Kemba Walker - Points\nKemba Walker Over (+23.5)\nKemba Walker Under (+23.5)\n1.90\n1.83',
 'Marvin Williams - Points\nMarvin Williams Over (+11.5)\nMarvin Williams Under (+11.5)\n1.78\n1.96',
 'Nicolas Batum - Points\nNicolas Batum Over (+8.5)\nNicolas Batum Under (+8.5)\n1.91\n1.82',
 'Nikola Jokic - Points\nNikola Jokic Over (+20.5)\nNikola Jokic Under (+20.5)\n1.99\n1.76',
 'Jamal Murray - Points\nJamal Murray Over (+19.5)\nJamal Murray Under (+19.5)\n1.74\n2.01',
 'Gary Harris - Points\nGary Harris Over (+15.5)\nGary Harris Under (+15.5)\n1.91\n1.82',
 'Paul Millsap - Points\nPaul Millsap Over (+11.5)\nPaul Millsap Under (+11.5)\n2.04\n1.72',
 'Kemba Walker - Rebounds\nKemba Walker Over (+3.5)\nKemba Walker Under (+3.5)\n1.61\n2.22',
 'Marvin Williams - Rebounds\nMarvin Williams Over (+6.5)\nMarvin Williams Under (+6.5)\n1.93\n1.80',
 'Nicolas Batum - Rebounds\nNicolas Batum Over (+5.5)\nNicolas Batum Under (+5.5)\n2.01\n1.74',
 'Nikola Jokic - Rebounds\nNikola Jokic Over (+11.5